In [4]:
# Check Current Working Directory
import os
print(os.getcwd())

S:\Projects\ABC\y2025\Pro\Draft\ABC_RSR_Calculation_Pro\ABC_RSR_Calculation


In [3]:
# Change working directory to the notebook's directory if needed 
# This ensures the config.py file in the same directory as the notebook can be referenced
os.chdir(r'S:\Projects\ABC\y2025\Pro\Draft\ABC_RSR_Calculation_Pro\ABC_RSR_Calculation')

In [6]:
# Module search path: Also ensures that the directory containing 'config.py' is in the Python search path ('sys.path').
import sys
sys.path.append(r'S:\Projects\ABC\y2025\Pro\Draft\ABC_RSR_Calculation_Pro\ABC_RSR_Calculation')

In [7]:
import arcpy
import os
import config
import numpy as np

Environments set from config file.
Paths set from config file.
File and variable names set from config file.


In [8]:
workspace_final = os.path.join(config.project_folder_final, f"{config.gdb_final}.gdb")
workspace_int = os.path.join(config.project_folder_int, f"{config.gdb_int}.gdb")

In [9]:
arcpy.env.workspace = workspace_final

In [10]:
# Calculate the deciles for RSR ('DecileExtent')

feature_class = config.ABC_Analysis_Table
field = 'Subtype_RSR'  # The field you want to calculate deciles for
decile_field = 'DecileExtent'  # The new field to store the decile classification

# Add a field to store the deciles if it doesn't already exist
if not arcpy.ListFields(feature_class, decile_field):
    arcpy.AddField_management(feature_class, decile_field, 'DOUBLE')

# Get all the values from the field
values = [row[0] for row in arcpy.da.SearchCursor(feature_class, [field])]

if values:
    
    # Calculate deciles using numpy
    deciles = np.quantile(values, [.10, .20, .30, .40, .50, .60, .70, .80, .90])

    # Classify the values into deciles and update the feature class
    with arcpy.da.UpdateCursor(feature_class, [field, decile_field]) as cursor:
        for row in cursor:
            value = row[0]
            if value <= deciles[0]:
                row[1] = 1
            elif value <= deciles[1]:
                row[1] = 2
            elif value <= deciles[2]:
                row[1] = 3
            elif value <= deciles[3]:
                row[1] = 4
            elif value <= deciles[4]:
                row[1] = 5
            elif value <= deciles[5]:
                row[1] = 6
            elif value <= deciles[6]:
                row[1] = 7
            elif value <= deciles[7]:
                row[1] = 8
            elif value <= deciles[8]:
                row[1] = 9
            else:
                row[1] = 10
            cursor.updateRow(row)
            
    print("Decile classification complete.")
else:
    print("No valid numeric data found in the field. Decile classification skipped.")

Decile classification complete.


In [11]:
# Calculate the deciles for Protection ('DecileProtection')
# Override the DecileProtection value for Rice to be 1
# Override the DecileProtection value for Polar Desert to be 10 - no longer
# relevant as Polar Desert was reclassified in a map update

# Define feature class and fields
feature_class = config.ABC_Analysis_Table
field = 'Protected_Area_1_2'  # The field you want to calculate deciles for
decile_field = 'DecileProtection'  # The new field to store the decile classification
habitat_code_field = 'HabitatCod'  # Field to identify HabitatCod

# HabitatCod overrides
override_values = {
    128445: 1,  # Set DecileProtection to 1
    95500: 10   # Set DecileProtection to 10
}

# Step 1: Add a field to store the deciles if it doesn't already exist
if not arcpy.ListFields(feature_class, decile_field):
    arcpy.AddField_management(feature_class, decile_field, 'DOUBLE')
    print(f"Added field '{decile_field}' to the table.")

# Step 2: Get all the values from the field and filter out None values
values = [row[0] for row in arcpy.da.SearchCursor(feature_class, [field]) if row[0] is not None]

# Calculate deciles using numpy (we will invert this logic)
deciles = np.quantile(values, [.10, .20, .30, .40, .50, .60, .70, .80, .90])

# Step 3: Classify the values into inverse deciles and update the feature class
with arcpy.da.UpdateCursor(feature_class, [field, decile_field, habitat_code_field]) as cursor:
    for row in cursor:
        value = row[0]
        habitat_cod = row[2]

        # Check for HabitatCod overrides
        if habitat_cod in override_values:
            row[1] = override_values[habitat_cod]  # Set override value
        elif value is None:
            row[1] = None  # Handle None values (or assign a default if needed)
        else:
            # Inverse decile classification: higher values go to lower decile
            if value >= deciles[8]:
                row[1] = 1
            elif value >= deciles[7]:
                row[1] = 2
            elif value >= deciles[6]:
                row[1] = 3
            elif value >= deciles[5]:
                row[1] = 4
            elif value >= deciles[4]:
                row[1] = 5
            elif value >= deciles[3]:
                row[1] = 6
            elif value >= deciles[2]:
                row[1] = 7
            elif value >= deciles[1]:
                row[1] = 8
            elif value >= deciles[0]:
                row[1] = 9
            else:
                row[1] = 10
        
        cursor.updateRow(row)

print("DecileProtection classification complete with overrides.")

Added field 'DecileProtection' to the table.
DecileProtection classification complete with overrides.


In [12]:
# Calculate the deciles for LCVC ('DecileConversion')

feature_class = config.ABC_Analysis_Table
field = 'Conversion_Mean'  # The field you want to calculate deciles for
decile_field = 'DecileConversion'  # The new field to store the decile classification

# Add a field to store the deciles if it doesn't already exist
if not arcpy.ListFields(feature_class, decile_field):
    arcpy.AddField_management(feature_class, decile_field, 'DOUBLE')

# Get all the values from the field
values = [row[0] for row in arcpy.da.SearchCursor(feature_class, [field])]

# Calculate deciles using numpy
deciles = np.quantile(values, [.10, .20, .30, .40, .50, .60, .70, .80, .90])

# Classify the values into deciles and update the feature class
with arcpy.da.UpdateCursor(feature_class, [field, decile_field]) as cursor:
    for row in cursor:
        value = row[0]
        if value <= deciles[0]:
            row[1] = 1
        elif value <= deciles[1]:
            row[1] = 2
        elif value <= deciles[2]:
            row[1] = 3
        elif value <= deciles[3]:
            row[1] = 4
        elif value <= deciles[4]:
            row[1] = 5
        elif value <= deciles[5]:
            row[1] = 6
        elif value <= deciles[6]:
            row[1] = 7
        elif value <= deciles[7]:
            row[1] = 8
        elif value <= deciles[8]:
            row[1] = 9
        else:
            row[1] = 10
        cursor.updateRow(row)
    
print("Decile classification complete.")

Decile classification complete.


In [13]:
# Calculate the deciles for Condition ('DecileCondition')
# Override the DecileCondition value for Rice to be 1

# Set the feature class and fields
feature_class = config.ABC_Analysis_Table
field = 'Condition_Mean'  # The field you want to calculate deciles for
decile_field = 'DecileCondition'  # The new field to store the decile classification
habitat_code_field = 'HabitatCod'  # Field to identify HabitatCod
override_habitat_cod = 128445  # HabitatCod for override
override_decile_value = 1  # Value to override DecileCondition

# Step 1: Add a field to store the deciles if it doesn't already exist
if not arcpy.ListFields(feature_class, decile_field):
    arcpy.AddField_management(feature_class, decile_field, 'DOUBLE')
    print(f"Added field '{decile_field}' to the table.")

# Step 2: Get all the values from the field
values = [row[0] for row in arcpy.da.SearchCursor(feature_class, [field]) if row[0] is not None]

# Calculate deciles using numpy (we will invert this logic)
deciles = np.quantile(values, [.10, .20, .30, .40, .50, .60, .70, .80, .90])

# Step 3: Classify the values into inverse deciles and update the feature class
with arcpy.da.UpdateCursor(feature_class, [field, decile_field, habitat_code_field]) as cursor:
    for row in cursor:
        value = row[0]
        habitat_cod = row[2]

        # Check for the override condition
        if habitat_cod == override_habitat_cod:
            row[1] = override_decile_value  # Override DecileCondition with 1
        else:
            # Inverse decile classification: higher values go to lower decile
            if value >= deciles[8]:
                row[1] = 1
            elif value >= deciles[7]:
                row[1] = 2
            elif value >= deciles[6]:
                row[1] = 3
            elif value >= deciles[5]:
                row[1] = 4
            elif value >= deciles[4]:
                row[1] = 5
            elif value >= deciles[3]:
                row[1] = 6
            elif value >= deciles[2]:
                row[1] = 7
            elif value >= deciles[1]:
                row[1] = 8
            elif value >= deciles[0]:
                row[1] = 9
            else:
                row[1] = 10
        
        cursor.updateRow(row)

print("Inverse decile classification complete.")

Added field 'DecileCondition' to the table.
Inverse decile classification complete.


In [14]:
# Calculate the deciles for Climate Velocity ('DecileClimate')

feature_class = config.ABC_Analysis_Table
field = 'Climate_Mean'  # The field you want to calculate deciles for
decile_field = 'DecileClimate'  # The new field to store the decile classification

# Add a field to store the deciles if it doesn't already exist
if not arcpy.ListFields(feature_class, decile_field):
    arcpy.AddField_management(feature_class, decile_field, 'DOUBLE')

# Get all the values from the field
values = [row[0] for row in arcpy.da.SearchCursor(feature_class, [field])]

# Calculate deciles using numpy
deciles = np.quantile(values, [.10, .20, .30, .40, .50, .60, .70, .80, .90])

# Classify the values into deciles and update the feature class
with arcpy.da.UpdateCursor(feature_class, [field, decile_field]) as cursor:
    for row in cursor:
        value = row[0]
        if value <= deciles[0]:
            row[1] = 1
        elif value <= deciles[1]:
            row[1] = 2
        elif value <= deciles[2]:
            row[1] = 3
        elif value <= deciles[3]:
            row[1] = 4
        elif value <= deciles[4]:
            row[1] = 5
        elif value <= deciles[5]:
            row[1] = 6
        elif value <= deciles[6]:
            row[1] = 7
        elif value <= deciles[7]:
            row[1] = 8
        elif value <= deciles[8]:
            row[1] = 9
        else:
            row[1] = 10
        cursor.updateRow(row)
    
print("Decile classification complete.")

Decile classification complete.


In [17]:
# Calculate the deciles for Obligation ('DecileObligation')

feature_class = config.ABC_Analysis_Table
field = 'Raw_Obligation_scores'  # The field you want to calculate deciles for
decile_field = 'DecileObligation'  # The new field to store the decile classification

# Add a field to store the deciles if it doesn't already exist
if not arcpy.ListFields(feature_class, decile_field):
    arcpy.AddField_management(feature_class, decile_field, 'DOUBLE')

# Get all the values from the field
values = [row[0] for row in arcpy.da.SearchCursor(feature_class, [field])]

# Calculate deciles using numpy
deciles = np.quantile(values, [.10, .20, .30, .40, .50, .60, .70, .80, .90])

# Classify the values into deciles and update the feature class
with arcpy.da.UpdateCursor(feature_class, [field, decile_field]) as cursor:
    for row in cursor:
        value = row[0]
        if value <= deciles[0]:
            row[1] = 1
        elif value <= deciles[1]:
            row[1] = 2
        elif value <= deciles[2]:
            row[1] = 3
        elif value <= deciles[3]:
            row[1] = 4
        elif value <= deciles[4]:
            row[1] = 5
        elif value <= deciles[5]:
            row[1] = 6
        elif value <= deciles[6]:
            row[1] = 7
        elif value <= deciles[7]:
            row[1] = 8
        elif value <= deciles[8]:
            row[1] = 9
        else:
            row[1] = 10
        cursor.updateRow(row)
    
print("Decile classification complete.")

Decile classification complete.


In [18]:
# Calculate the deciles for Bird Conservation ('DecileConservation')

feature_class = config.ABC_Analysis_Table
field = 'Raw_conservation_Concern_scores'  # The field you want to calculate deciles for
decile_field = 'DecileConservation'  # The new field to store the decile classification

# Add a field to store the deciles if it doesn't already exist
if not arcpy.ListFields(feature_class, decile_field):
    arcpy.AddField_management(feature_class, decile_field, 'DOUBLE')

# Get all the values from the field
values = [row[0] for row in arcpy.da.SearchCursor(feature_class, [field])]

# Calculate deciles using numpy
deciles = np.quantile(values, [.10, .20, .30, .40, .50, .60, .70, .80, .90])

# Classify the values into deciles and update the feature class
with arcpy.da.UpdateCursor(feature_class, [field, decile_field]) as cursor:
    for row in cursor:
        value = row[0]
        if value <= deciles[0]:
            row[1] = 1
        elif value <= deciles[1]:
            row[1] = 2
        elif value <= deciles[2]:
            row[1] = 3
        elif value <= deciles[3]:
            row[1] = 4
        elif value <= deciles[4]:
            row[1] = 5
        elif value <= deciles[5]:
            row[1] = 6
        elif value <= deciles[6]:
            row[1] = 7
        elif value <= deciles[7]:
            row[1] = 8
        elif value <= deciles[8]:
            row[1] = 9
        else:
            row[1] = 10
        cursor.updateRow(row)
    
print("Decile classification complete.")

Decile classification complete.


In [19]:
# Import necessary modules
import arcpy
import math  # For rounding up decimals

# Define the feature class or table and the fields you will use
feature_class = config.ABC_Analysis_Table  

# Define the fields to process and their weights
field_weights = {
    'DecileExtent': 2.0,         # Double
    'DecileProtection': 1.5,    # Increase by 1.5
    'DecileConversion': 1.5,    # Increase by 1.5
    'DecileCondition': 1.0,     # No weight
    'DecileClimate': 1.0,       # No weight
    'DecileObligation': 1.0,    # No weight
    'DecileConservation': 2.0   # Double
}

# List of HabitatCod values for which ThreatScore should be voided
void_habitat_codes = [128600, 117638, 117637, 117639, 128500, 128200, 128300, 117701, 117702, 128400, 117900]

# Name of the fields
habitat_code_field = 'HabitatCod'  # Adjust the field name to match your dataset
threat_score_field = 'ThreatScore'
score_status_field = 'Score_Status'

# Step 1: Add the ThreatScore field to the table if it doesn't already exist
if not arcpy.ListFields(feature_class, threat_score_field):
    arcpy.AddField_management(feature_class, threat_score_field, "DOUBLE")
    print(f"Added field '{threat_score_field}' to the table.")

# Step 2: Add the Score_Status field to the table if it doesn't already exist
if not arcpy.ListFields(feature_class, score_status_field):
    arcpy.AddField_management(feature_class, score_status_field, "TEXT", field_length=20)
    print(f"Added field '{score_status_field}' to the table.")

# Step 3: Use an UpdateCursor to calculate and update the ThreatScore and Score_Status fields
fields = list(field_weights.keys()) + [habitat_code_field, threat_score_field, score_status_field]

with arcpy.da.UpdateCursor(feature_class, fields) as cursor:
    for row in cursor:
        # Check if HabitatCod is in the void list
        habitat_code = row[-3]  # Third-to-last field
        if habitat_code in void_habitat_codes:
            # Void the ThreatScore and set Score_Status to 'Unscored'
            row[-2] = None  # ThreatScore set to NULL
            row[-1] = "Unscored"
        else:
            # Calculate the weighted sum for ThreatScore
            threat_score = 0
            for i, field in enumerate(field_weights.keys()):
                value = row[i]  # Get the value for the field
                if value is not None:  # Skip None values
                    threat_score += value * field_weights[field]  # Apply the weight

            # Round up the threat score if it has a decimal
            row[-2] = math.ceil(threat_score)  # Assign rounded-up value to the ThreatScore field
            row[-1] = "Scored"  # Set Score_Status to 'Scored'
        
        cursor.updateRow(row)

print("ThreatScore and Score_Status fields updated successfully.")

Added field 'ThreatScore' to the table.
Added field 'Score_Status' to the table.
ThreatScore and Score_Status fields updated successfully.


Create a Watchlist Field to flag the highest 15% of threat scores as Red, the next highest 15% as Yellow, the unscored habitats as 'Not Applicable', and the rest as Green.

In [20]:
# Import necessary modules
import arcpy

# Define feature class and fields
feature_class = config.ABC_Analysis_Table
threat_score_field = 'ThreatScore'  # Field storing threat scores
watchlist_field = 'Watchlist'       # Field to categorize habitats

# Step 1: Add the Watchlist field if it doesn't already exist
if not arcpy.ListFields(feature_class, watchlist_field):
    arcpy.AddField_management(feature_class, watchlist_field, 'TEXT', field_length=20)
    print(f"Added field '{watchlist_field}' to the table.")

# Step 2: Retrieve threat scores and habitat codes
data = []
with arcpy.da.SearchCursor(feature_class, ['HabitatCod', threat_score_field]) as cursor:
    for row in cursor:
        data.append(row)

# Step 3: Identify thresholds for 'Red' and 'Yellow' categories
valid_scores = [row[1] for row in data if row[1] is not None]  # Filter out null scores
valid_scores.sort(reverse=True)

# Calculate thresholds for the top 15% and next 15%
n = len(valid_scores)
if n > 0:
    red_threshold_index = max(1, int(0.15 * n))  # Top 15% index (at least 1)
    yellow_threshold_index = max(1, int(0.30 * n))  # Top 30% index (includes next 15%)

    red_threshold = valid_scores[red_threshold_index - 1]
    yellow_threshold = valid_scores[yellow_threshold_index - 1]
else:
    red_threshold = None
    yellow_threshold = None

# Step 4: Update the Watchlist field
with arcpy.da.UpdateCursor(feature_class, ['HabitatCod', threat_score_field, watchlist_field]) as cursor:
    for row in cursor:
        habitat_cod = row[0]
        threat_score = row[1]

        # Assign Watchlist category
        if threat_score is None:
            row[2] = 'Not Applicable'
        elif threat_score >= red_threshold:
            row[2] = 'Red'
        elif threat_score >= yellow_threshold:
            row[2] = 'Yellow'
        else:
            row[2] = 'Green'

        cursor.updateRow(row)

print("Watchlist categorization complete.")

Added field 'Watchlist' to the table.
Watchlist categorization complete.


OLD - PULL IN DECILE STATS THAT WERE RUN BY CHRIS FOR CLIMATE VELOCITY.
LATER UPDATE THE SCRIPT TO JUST CALCULATE THAT HERE. ALSO OLD Threat Score Sum Script.

In [13]:
# Sum the decile values to get a threat score for each subtype

# Define the feature class or table and the fields you will use
feature_class = config.ABC_Analysis_Table  

# List the fields you want to sum for the ThreatScore
fields_to_sum = ['DecileExtent','DecileProtection','DecileConversion','DecileCondition', 'DecileClimate', 'DecileObligation', 'DecileConservation']  

# Name of the new field to store the sum (ThreatScore)
threat_score_field = 'ThreatScore'

# Step 1: Add the ThreatScore field to the table if it doesn't already exist
if not arcpy.ListFields(feature_class, threat_score_field):
    arcpy.AddField_management(feature_class, threat_score_field, "DOUBLE")
    print(f"Added field '{threat_score_field}' to the table.")

# Step 2: Update the ThreatScore field by summing values from the five specified fields
# Fields to include in the cursor: the five fields to sum + the new ThreatScore field
fields = fields_to_sum + [threat_score_field]

# Step 3: Use an UpdateCursor to calculate and update the ThreatScore field
with arcpy.da.UpdateCursor(feature_class, fields) as cursor:
    for row in cursor:
        # Sum the values from the five fields
        sum_of_fields = sum([val for val in row[:-1] if val is not None])  # Ignore None values during the summation
        row[-1] = sum_of_fields  # Assign the sum to the ThreatScore field
        cursor.updateRow(row)

print("ThreatScore calculation complete.")

Added field 'ThreatScore' to the table.
ThreatScore calculation complete.


In [33]:
# Populate the final analysis table with the Climate Velocity Decile scores.

# Define table names and join field
final_table = config.ABC_Analysis_Table
intermediate_table = "ZonalSt_fwvel731_ensemble_8gcm_370_2041_2070"
join_field = "HabitatCod"

# Step 1: Add a permanent field 'Conversion_Mean' in the final analysis table before applying the join
conversion_field = "DecileClimate"
arcpy.management.AddField(final_table, conversion_field, "DOUBLE")

# Step 2: Add the join
arcpy.management.AddJoin(final_table, join_field, intermediate_table, join_field)

# Step 3: Read values from the 'Decile' field in the intermediate table
fields = ["{}.DecileMean".format(intermediate_table), "{}.{}".format(final_table, conversion_field)]

# Use a search cursor to gather the MEAN values from the intermediate table
mean_values = []
with arcpy.da.SearchCursor(final_table, fields) as cursor:
    for row in cursor:
        mean_values.append(row[0])  # Store the 'MEAN' values

# Step 4: Remove the join before updating the table
arcpy.management.RemoveJoin(final_table)

# Step 5: Use an update cursor to populate the 'Conversion_Mean' field with the stored MEAN values
with arcpy.da.UpdateCursor(final_table, [conversion_field]) as cursor:
    for i, row in enumerate(cursor):
        row[0] = mean_values[i]  # Assign the stored 'MEAN' value to the 'Conversion_Mean' field
        cursor.updateRow(row)

print("Table updated.")

Table updated.
